In [2]:
import pandas as pd
import numpy as np
import tables
import datetime as dt


exchange = 'NSE'
symbol = "HDFC"
today = '05_04_2019'#dt.datetime.now().strftime("%d_%m_%y")
sample_freq = '5min'

# OAuth Login

In [2]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

#logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="b2w0sfnr1zr92nxm")
#kite.login_url()

if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
else:
    driver = webdriver.Chrome("./automation/chromedriver")

driver.get(kite.login_url())

In [3]:
# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.
request_token = re.findall("request_token=(.*)&action",driver.current_url)

data = kite.generate_session(request_token[0], api_secret="jtga2mp2e5fn29h8w0pe2kb722g3dh1q")
kite.set_access_token(data["access_token"])

driver.close()

# Stock List Download

In [4]:
def getInstruments(exchange='NSE'):
    instruments_df = pd.DataFrame(data=kite.instruments(exchange))
    instruments_df = instruments_df.set_index('tradingsymbol')
    return instruments_df

In [5]:
instruments_df = getInstruments(exchange)
instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange+today,mode='a')

# Historical Data Download

In [6]:
def downloadData(symbol="HDFC", fromDate= dt.datetime.now() - dt.timedelta(days = 1), toDate=dt.datetime.now(), freq="minute"):
    symbolToken = instruments_df.loc[symbol,'instrument_token']
    raw_data = pd.DataFrame(data=kite.historical_data(symbolToken, fromDate, toDate, freq, continuous=False))
    raw_data = raw_data.set_index('date').tz_localize(None)
    return raw_data
                

In [7]:
def resample(data,freq):
    data = data.resample(freq).agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'})
    #data.columns = data.columns.droplevel()
    return data
    
    

In [8]:
raw_data = downloadData()

raw_data.to_hdf('kite_data/kite_data.h5', key=symbol+today,mode='a')

In [3]:
raw_data = pd.read_hdf('kite_data/kite_data.h5', key=symbol+"05_04_19",mode='r')

# Filter Stocks

# Live Data resampling

# Indicators calculation

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

lrange = raw_data.close[-1]-30
rrange = raw_data.close[-1]+30
rangeMin = raw_data.index[-100]
rangeMax = raw_data.index[-1]

#################################
#   Plotting Data              #
###############################
init_notebook_mode(connected=True)

In [153]:
from talib import MACD
from talib import RSI, BBANDS, MACD, AROON, STOCH, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX
from talib import CDLABANDONEDBABY, CDL3BLACKCROWS,CDLDOJI, CDLDOJISTAR, CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR, CDLGRAVESTONEDOJI, CDLHAMMER, CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLINVERTEDHAMMER,CDLMARUBOZU,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLSPINNINGTOP,CDL3BLACKCROWS, CDL3LINESTRIKE


def candlestick(price, pos=1, plot=False):
    # Candlestick
    trace = go.Candlestick(x=price.index, open=price['open'], high=price['high'], low=price['low'], close=price['close'], name="Candlestick", showlegend=False)

    if plot:
        fig.append_trace(trace, pos, 1)
    return price

def macd(price, pos=1, plot=False):
    price['macd'], price['macdsignal'], price['macdhist'] = MACD(price.close, fastperiod=12, slowperiod=26, signalperiod=9)
    
    # MACD plots
    traceMACD = go.Scatter(x=price.index, y=price.macd, name='MACD', line=dict(color='black'),showlegend=False)
    traceMACDSignal = go.Scatter(x=price.index, y=price.macdsignal, name='MACD signal', line=dict(color='red'),showlegend=False)
    traceMACDHist = go.Bar(x=price.index, y=price.macdhist, name='MACD Hist', marker=dict(color="grey"),showlegend=False)
        
    if plot:
        fig.append_trace(traceMACD, pos, 1)
        fig.append_trace(traceMACDSignal, pos, 1)
        fig.append_trace(traceMACDHist, pos, 1)
    
    return price

def rsi(price, pos=1, plot=False):
    price['RSI'] = RSI(price.close, timeperiod=14)
    
    traceRSI = go.Scatter(x=price.index, y=price['RSI'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI',showlegend=False)
    
    if plot:
        fig.append_trace(traceRSI, pos, 1)
    return price

def stoch(price, pos=1, plot=False):
    price['slowk'], price['slowd'] = STOCH(price.high, price.low, price.close)
    
    traceSK  = go.Scatter(x=price.index, y=price['slowk'], name='%K', line=dict(color='black'), yaxis='y3',showlegend=False)
    traceSD  = go.Scatter(x=price.index, y=price['slowd'], name='%D', line=dict(color='red'),showlegend=False)
    
    if plot:
        fig.append_trace(traceSK, pos, 1)
        fig.append_trace(traceSD, pos, 1)
    return price

def aroon(price, pos=1, plot=False):
    price['aroondown'], price['aroonup'] = AROON(price.high, price.low)
    
    traceARD  = go.Scatter(x=price.index, y=price['aroondown'], name='AROON Down', line=dict(color='red'), yaxis='y4',showlegend=False)
    traceARU  = go.Scatter(x=price.index, y=price['aroonup'], name='AROON UP', line=dict(color='green'),showlegend=False)
    
    if plot:
        fig.append_trace(traceARD, pos, 1)
        fig.append_trace(traceARU, pos, 1)
    return price

def atr(price, pos=1, plot=False, multiplier=2):
    price['atr'] = ATR(price.high, price.low, price.close, timeperiod=22)
    price['chandlierLong'] = price.close-multiplier*price.atr
    price['chandlierShort'] = price.close+multiplier*price.atr
    
    traceATR = go.Scatter(x=price.index, y=price['atr'],mode='lines', line=dict(color='lightgrey'), name='ATR',showlegend=False)
    
    traceStopLoss = go.Scatter(x=price.index, y=price['chandlierLong'].shift(1), mode='lines', line=dict(color='lightgrey'), name='SL',showlegend=False)
    if plot:
        fig.append_trace(traceStopLoss, pos, 1)
        
    return price

def bbands(price, pos=1, plot=False, plotPrice=False):
    price['bbup'], price['bbmid'], price['bblow'] = BBANDS(price.close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    
    tracePrice = go.Scatter(x=price.index, y=price.close, marker = dict(color='grey', size=2), mode='lines', name="Close Price", yaxis='y1', showlegend=False)
    traceBBU = go.Scatter(x=price.index, y=price['bbup'], name='BB_up',  line=dict(color='grey'),showlegend=False)
    traceBBL = go.Scatter(x=price.index, y=price['bblow'], name='BB_low',  line=dict(color='grey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
    traceBBM = go.Scatter(x=price.index, y=price.bbmid, name='BB_mid',  line=dict(color='grey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
    
    
    if plot:
        if plotPrice:
            fig.append_trace(tracePrice, pos, 1)
            
        fig.append_trace(traceBBU, pos, 1)
        fig.append_trace(traceBBL, pos, 1)
        fig.append_trace(traceBBM, pos, 1)
    
    return price

def obv(price, pos=1, plot=False):
    price['obv'] = OBV(np.linspace(1,100,price.index.shape[0]), price.volume)
    
    traceV = go.Bar(x=price.index, y=price.volume, name='Volume', marker=dict(color='lightgrey'),yaxis='y2',showlegend=False)
    traceOBV = go.Scatter(x=price.index, y=price['obv'], name='OBV', line=dict(color='black'),showlegend=False)
    return price

def adosc(price, pos=1, plot=False):
    
    price['adosc'] = ADOSC(price.high, price.low, price.close, price.volume)
    return price

def adx(price, pos=1, plot=False):
    price['ADX'] = ADX(price.high, price.low, price.close)
    price['DI_P'] = PLUS_DI(price.high, price.low, price.close)
    price['DI_M'] = MINUS_DI(price.high, price.low, price.close)
    
    traceADX  = go.Scatter(x=price.index, y=price['ADX'], name='ADX', line=dict(color='blue') ,showlegend=False)
    traceDIP  = go.Scatter(x=price.index, y=price['DI_P'], name='DI+', line=dict(color='green'),showlegend=False)
    traceDIM  = go.Scatter(x=price.index, y=price['DI_M'], name='DI-', line=dict(color='red'),showlegend=False)
    
    
    if plot:
        fig.append_trace(traceADX, pos, 1)
        fig.append_trace(traceDIP, pos, 1)
        fig.append_trace(traceDIM, pos, 1)
    return price

# Heikin-Ashi
def ha(price, pos=1, plot=False):
    price['HAopen'] = (price.open.shift(1) + price.close.shift(1))/2
    price['HAhigh'] = price[['high','open','close']].max(axis = 1, skipna = True)
    price['HAlow'] = price[['low','open','close']].min(axis = 1, skipna = True)
    price['HAclose'] = (price.open+price.high+price.low+price.close)/4
    
    traceHA = go.Candlestick(x=price.index, open=price['HAopen'], high=price['HAhigh'], low=price['HAlow'], close=price['HAclose'], name="Heikin Ashi", showlegend=False)
    
    if plot:
        fig.append_trace(traceHA, pos, 1)
    return price

def annotateBuySell(price):
    arr = []
    for index, row in price.iterrows():
        arr.append(dict(x=index,y=row['low']-0.3,
                                       xref='x',yref='y',showarrow=True,ax=0,ay=35,
                                       arrowhead=1,arrowsize=0.5, arrowwidth=7,arrowcolor='green'))
    fig['layout']['annotations']=arr
    return price
    

def createPlot():
    fig = tools.make_subplots(rows=4, cols=1, shared_xaxes=True, row_width=[1,1,1,5], vertical_spacing = 0)
    fig['layout']['xaxis'] = dict(rangeslider = dict(visible=False), side="top")
    fig['layout'].update(height=950, plot_bgcolor='rgba(0,0,0,0)', title="Charts for "+symbol)
    fig['layout']['yaxis']['anchor']='x'
    fig['layout']['xaxis']['rangeselector'] = dict(
                buttons=list([dict(count=1, label='1h', step='hour', stepmode='backward'),
                              dict(count=3, label='3h', step='hour', stepmode='backward'),
                              dict(count=6, label='1d', step='hour', stepmode='backward'),
                              dict(step='all')]))
    return fig




In [154]:
fig = createPlot()
raw_data = ha(raw_data,1,True)
raw_data = bbands(raw_data,1, True)
#raw_data = atr(raw_data,1, True, multiplier = 1.5)

raw_data = macd(raw_data,2,True)
raw_data = rsi(raw_data,3, True)
raw_data = aroon(raw_data,4, True)


annotateBuySell(raw_data.iloc[[100,150]])

iplot(fig, filename="analysis")



This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



# Algorithm

# Visualization

In [42]:
iplot(fig, filename="analysis")

# WebSocket Handler

In [ ]:
import multiprocessing

def worker():
    """worker function"""
    print 'Worker'
    return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker)
        jobs.append(p)
        p.start()